In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import numpy 
print(tf.__version__)

1.8.0


##  step 1: download the dataset

In [3]:
imdb = keras.datasets.imdb

In [4]:
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=10000)

### Lets explore the data we have..

In [13]:
print("Shape of training data: {}".format(train_data.shape))
print("shape of labels: {}".format(train_labels.shape))
print("Shape of training data: {}".format(test_data.shape))
print("shape of labels: {}".format(test_labels.shape))

Shape of training data: (25000,)
shape of labels: (25000,)
Shape of training data: (25000,)
shape of labels: (25000,)


In [5]:
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [6]:
train_labels[0]

1

In [7]:
## since the data form all represents the interger index we need to covert back our data into words 
word_index = imdb.get_word_index()
type(word_index)

dict

In [8]:
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

In [9]:
## here is the reverse dictionary
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [11]:
## lets decode the sentences 
def decode_review(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])

In [12]:
decode_review(train_data[0])

u"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for w

### Data cleansing step

In [16]:
## since the data in the set is 25000 we will not use one hot encoding method since it will make a 25,000 x 10,000 matrix
## it will be big so lets go with the padding approach. 
## padding will assign a max length for a each reviews and we will have the matrix comparatively smaller. 
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                     value=word_index["<PAD>"],
                                                     padding='post',
                                                     maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                    value=word_index["<PAD>"],
                                                     padding='post',
                                                     maxlen=256)

In [19]:
train_data.shape

(25000, 256)

In [20]:
train_data[0]

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223, 5244,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117, 5952,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

## Building the model 
Two basic questions:
1. How many layers?
2. How many hidden units?
input data consists of array of word indices we have to predict the labels whether its 0 or 1
so lets go with the model 


In [21]:
vocab_size = 10000

model = keras.Sequential()
## The resulting dimensions are: (batch, sequence, embedding).
## the input is a batch of input sequences, sequence is the number of length of sequencem, embedding is 16
model.add(keras.layers.Embedding(vocab_size, 16))
## the GlobalAveragePooling is to handle the variable length of the inputs like padding. 
model.add(keras.layers.GlobalAveragePooling1D())
## the number of hidden units= 16 we use relu here
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
## last layer contains only single hidden layer. 
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [22]:
## since its a simple classification problem 
## Generally binary_crossentropy is better dealing with the probabilities. 
## for the regression problems we will use the  mean_square_error funtion.
model.compile(optimizer=tf.train.AdamOptimizer(),
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [23]:
## lets create the validation set
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [24]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 1s 63us/step - loss: 0.6932 - acc: 0.5099 - val_loss: 0.6918 - val_acc: 0.5757
Epoch 2/40
15000/15000 [==============================] - 1s 47us/step - loss: 0.6909 - acc: 0.5582 - val_loss: 0.6898 - val_acc: 0.6069
Epoch 3/40
15000/15000 [==============================] - 1s 47us/step - loss: 0.6884 - acc: 0.5849 - val_loss: 0.6877 - val_acc: 0.5889
Epoch 4/40
15000/15000 [==============================] - 1s 42us/step - loss: 0.6852 - acc: 0.6261 - val_loss: 0.6854 - val_acc: 0.5148
Epoch 5/40
15000/15000 [==============================] - 1s 44us/step - loss: 0.6811 - acc: 0.6771 - val_loss: 0.6795 - val_acc: 0.7205
Epoch 6/40
15000/15000 [==============================] - 1s 47us/step - loss: 0.6748 - acc: 0.7323 - val_loss: 0.6727 - val_acc: 0.7256
Epoch 7/40
15000/15000 [==============================] - 1s 41us/step - loss: 0.6658 - acc: 0.7414 - val_loss: 0.6632 - val_acc

In [25]:
## evaluate the model 
results = model.evaluate(test_data, test_labels)

print(results)

25000/25000 [==============================] - 1s 36us/step
[0.3052408904647827, 0.87476]


## conclusion
This is a typical classification approach so advanced or lets say deep learning model can achieve upto 95% of the accuracy. 